## Requesting Annotations by Reference Designator

In this example we show how to request all annotations made for the Cabled Array International District Vent Field 2 (RS03INT2) Medium-Power JBox (MJ03D) Bottom Pressure and Tilt (06-BOTPTA303) instrument. The same technique applies to all instruments (Reference Designators) on the OOI.

In [1]:
import requests
import json
import netCDF4 as nc
import datetime
import pandas as pd

Below we have entered credential for a default user. These should work, unless they have been reset.  
Alternatively, you can login in at https://ooinet.oceanobservatories.org/ and obtain a API username and API token under your profile (top right corner).

In [2]:
username = 'OOIAPI-D8S960UXPK4K03'
token = 'IXL48EQ2XY'

Specify the reference designator you are looking for. For further information on finding reference designators for other instruments and be found via the data team portal at http://ooi.visualocean.net.

In [3]:
refdes = 'RS01SBPS-SF01A-4A-NUTNRA101'

The next step specifies to return all annotations between 2012 and now. Time stamps are converted from a string to milliseconds since 1970-01-01.

In [4]:
beginDT = int(nc.date2num(datetime.datetime.strptime("2012-01-01T01:00:01Z",'%Y-%m-%dT%H:%M:%SZ'),'seconds since 1970-01-01')*1000)
endDT = int(nc.date2num(datetime.datetime.utcnow(),'seconds since 1970-01-01')*1000)

Now we are ready to build the data request url and send the request.

In [5]:
anno_base_url = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find?' # base url and port for annotations

params = { # define parameters
    'beginDT':beginDT,
    'endDT':endDT,
    'refdes':refdes,
}

r = requests.get(anno_base_url, params=params,auth=(username, token)) # send data request

data = pd.read_json(json.dumps(r.json())) # convert json response to pandas dataframe
data

,@class,annotation,beginDT,endDT,exclusionFlag,id,method,node,parameters,qcFlag,sensor,source,stream,subsite
0,.AnnotationRecord,A brief violent storm caused loss of power to ...,1516330801000,1.516388e+12,False,319,None,None,[],not_operational,None,friedrich.knuth@rutgers.edu,None,RS01SBPS
1,.AnnotationRecord,SF01A powered down on 3/21/2018 due to oil lea...,1521591180000,NaN,False,1369,None,SF01A,[],not_operational,None,None,None,RS01SBPS
2,.AnnotationRecord,This instrument was deployed with onboard proc...,1436394121974,1.454638e+12,False,1370,streamed,SF01A,"[2320, 315]",suspect,4A-NUTNRA101,friedrich.knuth@rutgers.edu,nutnr_a_sample,RS01SBPS
3,.AnnotationRecord,"Note: PFE down. HVPS1 MOV explosion, 800A brea...",1417981500000,1.418688e+12,False,1248,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
4,.AnnotationRecord,Note: PNWGP Portland <-> Seattle outage. Possi...,1420615920000,1.420618e+12,False,1250,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
5,.AnnotationRecord,Note: Intermittent partial data loss due to st...,1422662400000,1.423008e+12,False,1251,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
6,.AnnotationRecord,Note: PNWGP outage due to City of Seattle fibe...,1426947000000,1.426998e+12,False,1256,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
7,.AnnotationRecord,Note: Network issues due to fire that damaged ...,1434153600000,1.434386e+12,False,1259,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
8,.AnnotationRecord,Note: Four 1-minute outages between Portland a...,1452147000000,1.452150e+12,False,1266,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS
9,.AnnotationRecord,Note: Unexplained loss of power at Pittock Bui...,1468291980000,1.468295e+12,False,1276,None,None,[],None,None,knuth@marine.rutgers.edu,None,RS01SBPS


As mentioned above, time stamps from the annotations API are handled in milliseconds since 1970-01-01. Let's create a function to convert them back to human readable time stamps and convert the millisecond time stamps in the data frame.

In [6]:
def convert_time(time_stamp):
    try: 
        time_stamp = (int(time_stamp)) / 1000
        time_stamp = nc.num2date(time_stamp,'seconds since 1970-01-01')
    except:
        pass
    return time_stamp

# convert time stamps
data['beginDT'] = data['beginDT'].apply(convert_time)
data['endDT'] = data['endDT'].apply(convert_time)

Now, let's select and print all the annotations that were made at the parameter level, by excluding any empty parameter level annotations. In this case, there is only one annotation that meets this criteria.

In [7]:
param_anno = data[data['parameters'].str.len() != 0]

for i in range(len((param_anno['annotation'].values))):
    print('start time:', param_anno['beginDT'].iloc[i])
    print('end time:', param_anno['endDT'].iloc[i], '\n')
    print('affected parameters:', param_anno['parameters'].iloc[i], '\n')
    print(param_anno['annotation'].iloc[i],'\n')

start time: 2015-07-08 22:22:01.974000
end time: 2016-02-05 02:00:11.695000 

affected parameters: [2320, 315] 

This instrument was deployed with onboard processing set to freshwater mode leading to an invalid L1 Dissolved Nitrate Concentration data product. The L2 version of the data product is unaffected by that onboard setting as it is calculated using the L0 data with salinity and temperature from a co-located CTD. Redmine: 8659 



## Known issue 1: it is hard to know what parameters are affected, just based on being given the parameter PD number.
You can cross reference the PD numbers with preload as follows, but this is far from optimal. For this reason, and until a solution is implemented, it is advised to include the affected parameter names in the annotation whenever making an annotation at the parameter level.

In [8]:
preload = 'https://raw.githubusercontent.com/oceanobservatories/preload-database/master/csv/ParameterDefs.csv'
pre_l = pd.read_csv(preload)

In [9]:
pre_l[['name','id','description']][pre_l['id'] == 'PD2320']


,name,id,description
1815,nutnr_nitrogen_in_nitrate,PD2320,Nitrogen in Nitrate


In [10]:
pre_l[['name','id','description']][pre_l['id'] == 'PD315']

,name,id,description
225,nitrate_concentration,PD315,"Dissolved Nitrate Concentration, uncorrected f..."


## Known issue 2: annotation time stamps are returned as milliseconds since 1970, while data time stamps are returned as seconds since 1900? 
The system should use consistent time stamps for all data and metadata types. There is no Redmine ticket on this as of yet.